In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from numpy import genfromtxt
import math

# Importing Required Libraries

This section of the code imports all the necessary libraries needed throughout this assignment.

- **Numpy:** Imported as np, it is a fundamental package for scientific computing in Python. It provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.

- **load_iris:** This function from the `sklearn.datasets` module loads and returns the iris dataset (classification).

- **train_test_split:** This function from the `sklearn.model_selection` module splits the dataset into two pieces: a training set and a testing set. The model will be trained on the training set and then tested for accuracy on the testing set.

- **genfromtxt:** This function from the `numpy` module loads data from a text file, with missing values handled as specified. It will be used to load the ionosphere dataset.

- **math:** This module provides mathematical functions defined by the C standard. It may be used in this assignment to handle operations involving infinity.

In [2]:
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Loading Iris Dataset

This code loads the Iris dataset and assigns it to the variable `iris` using the `load_iris()` function.

- **X_iris:** This variable holds the feature matrix for the Iris dataset. It is assigned the `data` attribute of the iris dataset. This matrix includes measurements for 150 iris flowers from three different species.

- **y_iris:** This variable holds the target vector for the Iris dataset. It is assigned the `target` attribute of the iris dataset. It includes the species of each iris flower, encoded as integers from 0 to 2.

In [3]:
ionosphere = genfromtxt('ionosphere.txt', delimiter=',')
X_ionosphere = ionosphere[:,:-1]  # All rows and all columns except the last one
y_ionosphere = ionosphere[:,-1]   # All rows and only the last column

# Loading Ionosphere Dataset

This code loads the Ionosphere dataset from a text file named 'ionosphere.txt' using the `genfromtxt()` function from numpy and assigns it to the variable `ionosphere`. The `delimiter=','` argument is used because the data in the file is comma-separated.

- **X_ionosphere:** This variable holds the feature matrix for the Ionosphere dataset. It is assigned all rows and all columns except the last one from the ionosphere dataset. The `[:,:-1]` indexing operation is used to select all rows and all columns excluding the last one.

- **y_ionosphere:** This variable holds the target vector for the Ionosphere dataset. It is assigned all rows and only the last column from the ionosphere dataset. The `[:,-1]` indexing operation is used to select all rows and only the last column.

In [4]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.33, random_state=2909)

# Splitting Iris Dataset

This line of code splits the Iris dataset into training and testing sets using the `train_test_split()` function from sklearn.

- **X_train_iris, X_test_iris:** These variables hold the feature matrices for the training and testing sets, respectively.

- **y_train_iris, y_test_iris:** These variables hold the target vectors for the training and testing sets, respectively.

The `test_size=0.33` argument specifies that 33% of the dataset will be used for the test set, and the rest (67%) will be used for the training set.

The `random_state=2909` argument is used for initializing the internal random number generator, which will decide the splitting of data into train and test indices. This ensures that the train and test splits generated are reproducible. The number `2909` is chosen based on my birthday (29th September) in DDMM format.

In [5]:
X_train_ionosphere, X_test_ionosphere, y_train_ionosphere, y_test_ionosphere = train_test_split(X_ionosphere, y_ionosphere, test_size=0.33, random_state=2909)

# Splitting Ionosphere Dataset

This line of code splits the Ionosphere dataset into training and testing sets using the `train_test_split()` function from sklearn.

- **X_train_ionosphere, X_test_ionosphere:** These variables hold the feature matrices for the training and testing sets, respectively.

- **y_train_ionosphere, y_test_ionosphere:** These variables hold the target vectors for the training and testing sets, respectively.

The `test_size=0.33` argument specifies that 33% of the dataset will be used for the test set, and the rest (67%) will be used for the training set.

The `random_state=2909` argument is used for initializing the internal random number generator, which will decide the splitting of data into train and test indices. This ensures that the train and test splits generated are reproducible. The number `2909` is chosen based on my birthday (29th September) in DDMM format.

In [6]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

# Defining Euclidean Distance Function

This function, `euclidean_distance(x1, x2)`, calculates the Euclidean distance between two points, `x1` and `x2`.

Euclidean distance is the straight line distance between two points in a space and can be calculated using the Pythagorean theorem. It is commonly used in machine learning to measure the similarity between vectors (in this case, `x1` and `x2`).

The function uses numpy's `sqrt()` function to calculate the square root, and the `sum()` function to calculate the sum of an array. The expression `(x1 - x2)**2` calculates the squared difference between corresponding elements in the `x1` and `x2` arrays. The `sum()` function then adds up these squared differences, and `sqrt()` is used to take the square root of this sum, resulting in the Euclidean distance.

In [7]:
def nearest_neighbour(X_train, y_train, x_test):
    # Calculate distances between x_test and all samples in X_train
    distances = [euclidean_distance(x_test, x_train) for x_train in X_train]

    # Get the index of the smallest distance
    nn_index = np.argmin(distances)

    # Return the corresponding class label
    return y_train[nn_index]

# Defining Nearest Neighbour Function

The function `nearest_neighbour(X_train, y_train, x_test)` is defined to implement the Nearest Neighbour algorithm. This algorithm is a type of instance-based learning or non-generalizing learning, where the function is approximated locally and all computation is deferred until function evaluation.

Step-by-step:

1. **Calculate Euclidean Distances:** The function computes the Euclidean distances between the test sample `x_test` and all samples in the training set `X_train`. This is achieved through a list comprehension which iteratively applies the `euclidean_distance` function to each training sample `x_train` in `X_train`.

2. **Identify Nearest Neighbour:** The function identifies the nearest neighbour by finding the index of the smallest distance in the `distances` list. This is performed using the numpy `argmin` function, which returns the indices of the minimum values along an axis.

3. **Return Class Label:** The function retrieves the corresponding class label from `y_train` for the identified nearest neighbour. This class label is the predicted label for the test sample `x_test`.

The overall goal of this function is to classify the test sample based on the class of its nearest neighbour in the feature space.

In [8]:
def test_nearest_neighbour(X_train, y_train, X_test, y_test):
    # Predict the class for each test instance
    y_pred = [nearest_neighbour(X_train, y_train, x_test) for x_test in X_test]

    # Calculate the error rate
    error_rate = np.sum(y_pred != y_test) / len(y_test)

    return error_rate

# Defining Test Function for Nearest Neighbour

The function `test_nearest_neighbour(X_train, y_train, X_test, y_test)` is defined to evaluate the performance of the Nearest Neighbour algorithm on the test set.

Step-by-step:

1. **Predict Classes:** The function predicts the class for each instance in the test set `X_test` by applying the `nearest_neighbour` function to each test sample. This is achieved through a list comprehension, resulting in a list of predicted class labels `y_pred`.

2. **Calculate Error Rate:** The function calculates the error rate as the proportion of test instances that were incorrectly classified. This is done by comparing the predicted class labels `y_pred` with the true class labels `y_test`, counting the number of mismatches, and dividing by the total number of test instances.

The function returns the calculated error rate, which provides a measure of how well the Nearest Neighbour algorithm performs on the test set.

In [9]:
iris_error_rate = test_nearest_neighbour(X_train_iris, y_train_iris, X_test_iris, y_test_iris)
print(f'Iris dataset error rate: {iris_error_rate * 100:.2f}%')

ionosphere_error_rate = test_nearest_neighbour(X_train_ionosphere, y_train_ionosphere, X_test_ionosphere, y_test_ionosphere)
print(f'Ionosphere dataset error rate: {ionosphere_error_rate * 100:.2f}%')

Iris dataset error rate: 8.00%
Ionosphere dataset error rate: 15.52%


# Evaluating Nearest Neighbour on Iris and Ionosphere Datasets

These lines of code evaluate the performance of the Nearest Neighbour algorithm on the Iris and Ionosphere datasets.

1. **Evaluate on Iris Dataset:** The function `test_nearest_neighbour` is called with the training and testing sets of the Iris dataset as arguments. This function returns the error rate for the Iris dataset, which is then stored in the variable `iris_error_rate`. The error rate is printed in percentage format with a precision of two decimal places.

2. **Evaluate on Ionosphere Dataset:** Similarly, the function `test_nearest_neighbour` is called with the training and testing sets of the Ionosphere dataset as arguments. The returned error rate is stored in the variable `ionosphere_error_rate` and printed in the same format as above.

The error rate provides a measure of how well the Nearest Neighbour algorithm performs on each dataset. Lower error rates indicate better performance.

In [10]:
def conformity_measure(X_train, y_train, x_test, y_test):
    same_class_indices = np.where(y_train == y_test)
    diff_class_indices = np.where(y_train != y_test)

    same_class_distances = [euclidean_distance(x_test, x_train) for x_train in X_train[same_class_indices]]
    diff_class_distances = [euclidean_distance(x_test, x_train) for x_train in X_train[diff_class_indices]]

    if len(same_class_distances) == 0:
        same_class_nearest_distance = math.inf
    else:
        same_class_nearest_distance = np.min(same_class_distances)

    if len(diff_class_distances) == 0:
        diff_class_nearest_distance = math.inf
    else:
        diff_class_nearest_distance = np.min(diff_class_distances)

    if same_class_nearest_distance == 0:
        return math.inf
    else:
        return diff_class_nearest_distance / same_class_nearest_distance

# Defining Conformity Measure Function

The function `conformity_measure(X_train, y_train, x_test, y_test)` is defined to compute the conformity measure for the Nearest Neighbour algorithm. The conformity measure is used in the conformal prediction framework to quantify how well a new example conforms to a set of previously seen examples.

Step-by-step:

1. **Identify Same and Different Class Indices:** The function identifies the indices of training instances that belong to the same class as the test instance (`same_class_indices`) and those that belong to a different class (`diff_class_indices`).

2. **Calculate Class Distances:** The function calculates the Euclidean distances from the test instance to the training instances of the same class (`same_class_distances`) and to those of different classes (`diff_class_distances`).

3. **Determine Nearest Distances:** The function determines the nearest distances among the same class distances (`same_class_nearest_distance`) and the different class distances (`diff_class_nearest_distance`). If there are no distances to consider (i.e., the list is empty), the nearest distance is set to infinity.

4. **Calculate Conformity Measure:** The function calculates the conformity measure as the ratio of the nearest different class distance to the nearest same class distance. If the nearest same class distance is zero, the conformity measure is set to infinity.

The function returns the calculated conformity measure, which provides a quantification of how similar the test instance is to instances of the same class compared to instances of different classes.

In [11]:
def nearest_neighbour_conformal_predictor(X_train, y_train, X_test, y_test):
    # Calculate the conformity measures for all test instances
    alphas = [conformity_measure(X_train, y_train, x_test, y_test) for x_test, y_test in zip(X_test, y_test)]

    # Calculate the p-values for all possible labels for all test instances
    p_values = [np.mean([1 if alpha >= alpha_prime else 0 for alpha_prime in alphas]) for alpha in alphas]

    # Calculate the average false p-value
    avg_false_p_value = np.mean([p for p, y in zip(p_values, y_test) if p < y])

    return avg_false_p_value

# Defining Nearest Neighbour Conformal Predictor Function

The function `nearest_neighbour_conformal_predictor(X_train, y_train, X_test, y_test)` is defined to implement the Nearest Neighbour Conformal Predictor. Conformal prediction is a technique for providing a measure of certainty for machine learning predictions.

Step-by-step:

1. **Calculate Conformity Measures:** The function calculates the conformity measures for all test instances. This is done by applying the `conformity_measure` function to each test sample and its corresponding label. The resulting conformity measures are stored in the `alphas` list.

2. **Calculate P-values:** The function calculates the p-values for all possible labels for all test instances. The p-value for a test instance is the proportion of conformity measures from the training set that are lower than or equal to the conformity measure of the test instance. The p-values are stored in the `p_values` list.

3. **Calculate Average False P-value:** The function calculates the average false p-value, which is the mean of the p-values for test instances that were incorrectly classified. This is done by selecting the p-values corresponding to test instances where the predicted label is less than the true label, and calculating the mean of these selected p-values.

The function returns the average false p-value, which provides a measure of the error rate of the conformal predictor. Lower average false p-values indicate better performance.

In [12]:
iris_avg_false_p_value = nearest_neighbour_conformal_predictor(X_train_iris, y_train_iris, X_test_iris, y_test_iris)
print(f'Iris dataset average false p-value: {iris_avg_false_p_value * 100:.2f}%')

ionosphere_avg_false_p_value = nearest_neighbour_conformal_predictor(X_train_ionosphere, y_train_ionosphere, X_test_ionosphere, y_test_ionosphere)
print(f'Ionosphere dataset average false p-value: {ionosphere_avg_false_p_value * 100:.2f}%')

Iris dataset average false p-value: 32.45%
Ionosphere dataset average false p-value: 64.80%


# Evaluating Average False P-value on Iris and Ionosphere Datasets

This code evaluates the average false p-value of the Nearest Neighbour Conformal Predictor on the Iris and Ionosphere datasets.

1. **Evaluate on Iris Dataset:** The function `nearest_neighbour_conformal_predictor` is called with the training and testing sets of the Iris dataset as arguments. This function returns the average false p-value for the Iris dataset, which is then stored in the variable `iris_avg_false_p_value`. The average false p-value is printed in percentage format with a precision of two decimal places.

2. **Evaluate on Ionosphere Dataset:** Similarly, the function `nearest_neighbour_conformal_predictor` is called with the training and testing sets of the Ionosphere dataset as arguments. The returned average false p-value is stored in the variable `ionosphere_avg_false_p_value` and printed in the same format as above.

The average false p-value provides a measure of the error rate of the conformal predictor. Lower average false p-values indicate better performance.